In [1]:
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np
from rembg import remove
from PIL import Image

ModuleNotFoundError: No module named 'onnxruntime'

# Fragrance Recommendation System

## Overview
This script extracts dominant colors from an image and recommends perfumes based on:
- **Color detection** → Matches colors with olfactive accords.
- **Situation-based filtering** → Suggests fragrances based on context (e.g., "formal event").
- **Preference filtering** → Recommends perfumes containing a specific accord.
- **Exclusion filtering** → Avoids perfumes with unwanted accords.
- **Gender filtering** → Returns perfumes based on gender (`Male`, `Female`, `Unisex`).


## Step 1: Load the Fragrance Database
This loads the dataset containing perfume information, including olfactive profiles and gender classification.

In [2]:
#Reading the database created for the fragrance recomendation system
fragrance_df = pd.read_csv("../data/fragrance_ML_model.csv")

## Step 2: Define fuction to remove the background from the image

In [3]:
def remove_background(input_path, output_path):
    """Removes background from an image using rembg."""
    image = Image.open(input_path)
    output = remove(image)
    output.save(output_path)
    print(f" Background removed successfully! Saved as {output_path}")
    output.show()

## Step 3: Extract Dominant Colors from uploaded image

In [5]:
def extract_dominant_colors(image_path, num_colors=3):
    """
    Extracts the most dominant colors from an image.
    """
    image = Image.open(image_path).convert("RGB")
    image = image.resize((100, 100))  # Reduce size for performance
    
    image_np = np.array(image)
    pixels = image_np.reshape(-1, 3)  # Convert to a list of RGB pixels
    
    color_counts = Counter(map(tuple, pixels))
    dominant_colors = [color for color, _ in color_counts.most_common(num_colors)]
    
    return dominant_colors

## Step 3: Map Colors to Olfactive Accords

In [6]:
COLOR_TO_ACCORD = {
    (255, 255, 0): ["citrus", "fruity", "sweet"],  # Yellow
    (0, 0, 255): ["marine", "aquatic", "ozonic"],  # Blue
    (245, 245, 220): ["powdery", "soft spicy", "musky"],  # Beige
    (255, 255, 255): ["clean", "aldehyde", "soft spicy"],  # White
    (128, 128, 128): ["musky", "powdery", "earthy"],  # Gray
    (255, 0, 255): ["sweet", "fruity", "floral"],  # Magenta
    (139, 69, 19): ["woody", "amber", "leather"],  # Brown
    (210, 180, 140): ["warm spicy", "woody", "sweet"],  # Light Brown
    (192, 192, 192): ["aldehyde", "fresh", "clean"],  # Metallic
    (255, 165, 0): ["citrus", "spicy", "warm"],  # Orange
    (0, 0, 0): ["leather", "smoky", "dark"],  # Black
    (255, 0, 0): ["spicy", "warm", "oriental"],  # Red
    (255, 192, 203): ["floral", "powdery", "sweet"],  # Pink
    (0, 255, 0): ["fresh", "herbal", "green"]  # Green
}

def extract_dominant_colors(image_path, num_colors=3):
    """
    Extracts the most dominant colors from an image using K-means clustering.
    """
    image = Image.open(image_path).convert("RGB")
    image = image.resize((100, 100))
    
    image_np = np.array(image)
    pixels = image_np.reshape(-1, 3)
    
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=num_colors, random_state=0)
    kmeans.fit(pixels)
    dominant_colors = kmeans.cluster_centers_.astype(int)
    
    return [tuple(color) for color in dominant_colors]

def map_colors_to_accords(dominant_colors):
    """Maps extracted colors to their corresponding olfactive accords."""
    accords = []
    for color in dominant_colors:
        closest_color = min(COLOR_TO_ACCORD.keys(), key=lambda c: np.linalg.norm(np.array(c) - np.array(color)))
        accords.extend(COLOR_TO_ACCORD[closest_color])
    return list(set(accords))

## Step 3: Define the Recommendation Function
This function processes the image, extracts dominant colors, matches them with accords, and filters perfumes based on the given criteria.
How filtering works:
1. **`preference`** → Filters perfumes that **contain** a specific scent profile in the `Olfactive Profile` column.
2. **`exclude`** → Removes perfumes that **contain** a specific scent profile in the `Olfactive Profile` column.
3. **`gender`** → Filters perfumes based on the `Gender` column (`Male`, `Female`, `Unisex`).

In [7]:
SITUATION_TO_ACCORD = {
    "formal": ["woody", "spicy", "leather"],
    "casual": ["fresh", "citrus", "aquatic"],
    "romantic": ["floral", "sweet", "musky"],
    "sport": ["green", "aquatic", "ozonic"],
    "office": ["clean", "powdery", "aldehyde"]
}

def recommend_fragrance(image_path, situation=None, preference=None, exclude=None, gender=None, num_recommendations=5):
    """
    Recommends perfumes based on extracted colors, situation, and filters.
    """
    segmented_image_path = remove_background(image_path)
    dominant_colors = extract_dominant_colors(segmented_image_path)
    accords_segmented = map_colors_to_accords(dominant_colors)
    
    combined_accords = accords_segmented[:]
    if situation in SITUATION_TO_ACCORD:
        combined_accords.extend(SITUATION_TO_ACCORD[situation])
    combined_accords = list(set(combined_accords))
    
    fragrance_df = pd.read_csv("../data/fragrance_ML_model.csv")
    filtered_df = fragrance_df[fragrance_df["Olfactive Profile"].str.contains('|'.join(combined_accords), case=False, na=False)]
    
    if gender:
        filtered_df = filtered_df[filtered_df["Gender"].str.contains(gender, case=False, na=False)]
    if preference:
        filtered_df = filtered_df[filtered_df["Olfactive Profile"].str.contains(preference, case=False, na=False)]
    if exclude:
        filtered_df = filtered_df[~filtered_df["Olfactive Profile"].str.contains(exclude, case=False, na=False)]
    
    if filtered_df.empty:
        return "No perfumes found after applying filters. Try adjusting your criteria."
    
    top_perfumes = filtered_df.sort_values(by=["Rating Value", "Rating Count"], ascending=[False, False])
    return top_perfumes.head(num_recommendations)

uploader = widgets.FileUpload(accept='.png,.jpg,.jpeg', multiple=False)
display(uploader)

FileUpload(value={}, accept='.png,.jpg,.jpeg', description='Upload')

In [1]:
if uploader.value:
    uploaded_file = list(uploader.value.values())[0]
    image_path = "test_image.jpg"
    with open(image_path, "wb") as f:
        f.write(uploaded_file['content'])
    print(f"Image saved as {image_path}")
else:
    print("No file uploaded! Please upload an image first.")


NameError: name 'uploader' is not defined

In [10]:
segmented_image_path = remove_background("test_image.jpg")

# Show original vs segmented image
original = Image.open("test_image.jpg")
segmented = Image.open(segmented_image_path)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(segmented)
plt.title("Segmented Image (No Background)")
plt.axis("off")

plt.show()

: 

## Step 4: Run a Test Example
This runs the function with a sample image, applying filters for situation, preference, exclusion, and gender.

In [ ]:
recommend_fragrance(
    "image.png",
    situation="formal event",  # Context-based recommendation
    preference="fresh",  # Keep only perfumes with "fresh"
    exclude="sweet",  # Remove perfumes with "sweet"
    gender="men",  # Filter by gender

SyntaxError: incomplete input (3431706157.py, line 6)

In [ ]:
def recommend_fragrance(image_path, situation=None, preference=None, exclude=None, gender=None, num_recommendations=5):
    """
    Recommends perfumes based on extracted colors from an image, a given situation, 
    user preferences, exclusions, and gender preference.
    """
    segmented_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    segmented_pil = Image.fromarray(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
    segmented_image_path = "segmented_image.png"
    segmented_pil.save(segmented_image_path)
    
    dominant_colors = extract_dominant_colors(segmented_image_path)
    accords_segmented = map_colors_to_accords(dominant_colors)
    
    # Combine extracted accords with situation-based accords
    combined_accords = accords_segmented[:]
    
    if situation and situation in SITUATION_TO_ACCORD:
        combined_accords.extend(SITUATION_TO_ACCORD[situation])
    
    combined_accords = list(set(combined_accords))
    
    # Filter perfumes based on olfactive profile
    filtered_df = fragrance_df[fragrance_df["Olfactive Profile"].str.contains('|'.join(combined_accords), case=False, na=False)]
    
    # Apply gender filter if specified
    if gender:
        filtered_df = filtered_df[filtered_df["Gender"].str.contains(gender, case=False, na=False)]
    
    # Apply preference filter if specified
    if preference:
        filtered_df_pref = filtered_df[filtered_df["Olfactive Profile"].str.contains(preference, case=False, na=False)]
        if not filtered_df_pref.empty:
            filtered_df = filtered_df_pref  # Use only if results are available
    
    # Apply exclusion filter if specified
    if exclude:
        filtered_df_exclude = filtered_df[~filtered_df["Olfactive Profile"].str.contains(exclude, case=False, na=False)]
        if not filtered_df_exclude.empty():
            filtered_df = filtered_df_exclude  # Use only if results are available
    
    if filtered_df.empty:
        return "No perfumes found after applying filters. Try adjusting your criteria."
    
    # Apply ranking and diversity
    top_perfumes = filtered_df.sort_values(by=["Rating Value", "Rating Count"], ascending=[False, False])
    if len(top_perfumes) > num_recommendations:
        diverse_recommendations = random.sample(top_perfumes.head(15).to_dict("records"), num_recommendations)
        return pd.DataFrame(diverse_recommendations)
    else:
        return top_perfumes.head(num_recommendations)


